# A anomalia de campo total

Nessa aula, vamos tentar entender a anomalia magnética de campo total:

$$
\Delta T = \|\mathbf{\vec{T}}\| - \|\mathbf{\vec{F}}\|
$$

na qual $\mathbf{\vec{T}}$ é o campo magnético observado e $\mathbf{\vec{F}}$ é o campo magnético interno (dado pelo IGRF).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import harmonica as hm
import verde as vd

## O campo magnético induzido em um cubo

Vamos ver como varia o campo magnético induzido em cubo para diferentes inclinações do campo magnético indutor.

Rode a célula abaixo para criar um cubo e produzir uma figura interativa.

In [2]:
def plota_cubo(ax, cubo, alpha=1):
    cubo = plt.Rectangle(
        (cubo[0], cubo[2]), cubo[1] - cubo[0], cubo[3] - cubo[2], facecolor="grey", alpha=alpha
    )
    ax.add_patch(cubo)
    
def anomalia_cubo_perfil(inclinação, magnetização):
    areacubo = [-400, 400, -1200, -400]
    cubo = (-50e3, 50e3, areacubo[0], areacubo[1], areacubo[2], areacubo[3])
    area = (-1200, 1200, -2000, 600)
    y, z, x = vd.grid_coordinates(area, spacing=200, extra_coords=0)
    def mask(area, v):
        y1, y2, z1, z2 = area
        v[(y >= y1) & (y <= y2) & (z >= z1) & (z <= z2)] = 0
        return v
    terra = hm.magnetic_angles_to_vec(10000, inclinação, 0)
    mag = hm.magnetic_angles_to_vec(magnetização, inclinação, 0)
    bx, by, bz = hm.prism_magnetic((x, y, z), cubo, mag, field="b")
    by = mask(areacubo, by)
    bz = mask(areacubo, bz)
    ty =  by + terra[1]
    tz =  bz + terra[2]
    mx, my, mz = mag
    # Cria a figura
    fig, axes = plt.subplots(1, 2, sharey='all', figsize=(10, 6), layout="constrained")
    ax1, ax2 = axes
    for ax in axes:
        ax.set_aspect('equal')
        ax.hlines(0, area[0], area[1], linewidth=3, zorder=1)
        ax.set_xlabel('y (m)')  
    ax1.quiver(y, z, by, bz, linewidth=0.5, scale=1e5, pivot='middle')
    ax2.quiver(
        y, z, ty, tz, 
        linewidth=0.5, 
        color='b',
        scale=1e5, 
        pivot='middle',
    )
    ax2.quiver(
        [-1000], [-0], [terra[1]], [terra[2]], 
        linewidth=0.5, 
        color='y',
        scale=1e5, 
        pivot='middle', 
    )
    ax1.set_ylabel('z (m)')
    ax1.set_title('Campo do corpo')  
    ax2.set_title('Campo total = corpo + Terra')  
    for ax in axes:
        plota_cubo(ax, areacubo)
        ax.quiver(
            [0], [-800], [my], [mz], 
            linewidth=0.5, 
            color='r',
            scale=2.5e2, 
            pivot='middle', 
        )
        ax.set_xlim(*area[:2])
        ax.set_ylim(*area[2:])
    
widgets.interactive(
    anomalia_cubo_perfil, 
    inclinação=widgets.IntSlider(min=-90, max=90, step=10, value=-40),
    magnetização=widgets.IntSlider(min=0, max=50, step=10, value=0, style={'description_width': 'initial'}),
)

interactive(children=(IntSlider(value=-40, description='inclinação', max=90, min=-90, step=10), IntSlider(valu…

## Tarefa: Desenhe a anomalia magnética do cubo para diferentes inclinações

Use a figura acima para deduzir e esboçar como seria a anomalia magnética do cupo (em perfil) para difentes inclinações: -90, -50, 0, 50, 90

## Anomalia magnética de um cubo

Agora vamos plotar um mapa da anomalia magnética do cubo para explorar a variação com a declinação e profundidade.

In [3]:
def anomalia_cubo_mapa(inclinação, declinação, magnetização, profundidade):
    area = (-15e3, 15e3, -10e3, 10e3)
    cubo = [-1e3, 1e3, -1e3, 1e3, -2e3, 0]
    x, y, z = vd.grid_coordinates(area, spacing=250, extra_coords=profundidade)
    fx, fy, fz = hm.magnetic_angles_to_vec(1, inclinação, declinação)
    mag = hm.magnetic_angles_to_vec(magnetização, inclinação, declinação)
    bx, by, bz = hm.prism_magnetic((x, y, z), cubo, mag, field="b")
    anomalia = fx*bx + fy*by + fz*bz
    scale = vd.maxabs(anomalia)
    # Cria a figura
    fig, ax = plt.subplots(1, 1, figsize=(10, 5.5), layout="constrained")
    ax.set_aspect('equal')  
    tmp = ax.pcolormesh(x, y, anomalia, cmap="RdBu_r", vmin=-scale, vmax=scale)
    fig.colorbar(tmp, ax=ax, label="nT", pad=0, aspect=40)
    plota_cubo(ax, cubo[:4], alpha=0.5)        
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_xlim(*area[:2])
    ax.set_ylim(*area[2:])
    
widgets.interactive(
    anomalia_cubo_mapa, 
    inclinação=widgets.IntSlider(min=-90, max=90, step=10, value=-40),
    declinação=widgets.IntSlider(min=-90, max=90, step=10, value=0),
    magnetização=widgets.IntSlider(min=0, max=20, step=5, value=0, style={'description_width': 'initial'}),
    profundidade=widgets.IntSlider(min=500, max=5000, step=500, value=500, style={'description_width': 'initial'}),
)

interactive(children=(IntSlider(value=-40, description='inclinação', max=90, min=-90, step=10), IntSlider(valu…